In [ ]:
import numpy as np
from qunetsim.components import Host
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.objects import Qubit

Logger.DISABLED = True

def encode_repetition(host, logical_qubit):
    #Encodes a logical qubit using the repetition code (3 qubits).
   
    q1 = Qubit(host)
    q2 = Qubit(host)
    q3 = Qubit(host)

    # Apply X gates if logical_qubit is 1
    if logical_qubit == 1:
        q1.X()
        q2.X()
        q3.X()
    return [q1, q2, q3]

def decode_repetition(qubits):

    #Decodes the qubit using majority voting.
    
    results = [q.measure() for q in qubits]
    print(f"Encoded bits: {results}")
    return 1 if results.count(1) > results.count(0) else 0


def protocol_1(host, receiver, designated_node):
    # Create EPR pair
    q1 = Qubit(host)
    q2 = Qubit(host)
    q1.H()
    q1.cnot(q2)
    host.x_error_rate = 0.8
    # Send one part of the EPR pair and encoded qubits to the receiver
    
    host.send_qubit(receiver, q2)
    q_measurement = q1.measure()
    host.send_classical(designated_node, str(q_measurement))
    
    encoded_qubits = encode_repetition(host, logical_qubit=q_measurement)  
      
    for q in encoded_qubits:
        if np.random.rand() < host.x_error_rate:
            q.X()
        host.send_qubit(receiver, q)

    # Measure one qubit in the EPR pair locally
    
    print(f"Measurement at Host 1 is: {q_measurement}")
    print(f"Host 1 sent EPR pair and encoded qubits to {receiver}")
    return encoded_qubits, q1

def protocol_2(host, sender, designated_node):
    # Receive the EPR pair and encoded qubits
    epr_qubit = host.get_data_qubit(sender, wait=5)
    encoded_qubits = [host.get_data_qubit(sender, wait=5) for _ in range(3)]
    if epr_qubit is not None and all(encoded_qubits):
        # Decode the repetition code
        decoded_value = decode_repetition(encoded_qubits)
        print(f"Decoded value at Host 2: {decoded_value}")
        host.send_classical(designated_node, str(decoded_value))
        print(f"Host 2 sent measurement result to {designated_node}")
    else:
        print("Host 2 did not receive all qubits.")
    
def designated_protocol(host, node_1, node_2):
    # Get the classical messages
    result1 = host.get_classical(node_1, wait=10)
    result2 = host.get_classical(node_2, wait=10)
    
    # Get the state of the qubits
    result1_content = int(result1[0].content)
    result2_content = int(result2[0].content)

    # Compute parity using XOR
    parity = result1_content ^ result2_content
    print(f"Computed parity at designated node: {parity}")

    if parity == 0:
        print("Parity check passed. No errors detected in logical bits.")
    else:
        print("Parity check failed. Errors might have occurred.")

def main():
    # Set up the network
    network = Network.get_instance()
    nodes = ['Alice', 'Bob', 'Eve']
    network.start(nodes)

    host_Alice = Host('Alice')
    host_Alice.add_connection('Bob')
    host_Alice.start()

    host_Bob = Host('Bob')
    host_Bob.add_connection('Alice')
    host_Bob.add_connection('Eve')
    host_Bob.start()

    host_Eve = Host('Eve')
    host_Eve.add_connection('Bob')
    host_Eve.start()

    network.add_host(host_Alice)
    network.add_host(host_Bob)
    network.add_host(host_Eve)

    # Alice and Eve are hosts; Bob is the designated node
    host_Alice.run_protocol(protocol_1, (host_Eve.host_id, host_Bob.host_id))
    host_Eve.run_protocol(protocol_2, (host_Alice.host_id, host_Bob.host_id))
    host_Bob.run_protocol(designated_protocol, ('Eve', 'Alice'))

if __name__ == '__main__':
    main()


Measurement at Host 1 is: 1
Host 1 sent EPR pair and encoded qubits to Eve
Encoded bits: [0, 0, 0]
Decoded value at Host 2: 0
Host 2 sent measurement result to Bob
Computed parity at designated node: 1
Parity check failed. Errors might have occurred.
